# Assignment 4
## Nakiyah Dhariwala


### Final Dataset Selection:
https://archive.ics.uci.edu/dataset/320/student+performance


In [1]:
!pip install git+https://github.com/MaximeJumelle/ALEPython.git@dev#egg=alepython

  Cloning https://github.com/MaximeJumelle/ALEPython.git (to revision dev) to /private/var/folders/f4/vybgzbbx1sg165hn_kn98j0m0000gn/T/pip-install-kcdfkda_/alepython_f1d1e00b86024bc9b1906b175a4c83e1
  Running command git clone --filter=blob:none --quiet https://github.com/MaximeJumelle/ALEPython.git /private/var/folders/f4/vybgzbbx1sg165hn_kn98j0m0000gn/T/pip-install-kcdfkda_/alepython_f1d1e00b86024bc9b1906b175a4c83e1
  Resolved https://github.com/MaximeJumelle/ALEPython.git to commit 286350ab674980a32270db2a0b5ccca1380312a7
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from alepython import ale_plot


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.inspection import PartialDependenceDisplay


# Set options to display all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

### Step 1: loading and viewing the dataset

I am just doing initial EDA to check for missing values, incorrect data types and to see what the data contains

In [3]:
# Loading the math dataset
df = pd.read_csv("student/student-mat.csv", sep=";")

FileNotFoundError: [Errno 2] No such file or directory: 'student/student-mat.csv'

In [ ]:
# Quick look
print("The dimensions of this dataset are:", df.shape)
print("The columns for this dataset are", df.columns)

In [ ]:
print(df.head())

In [ ]:
# Checking for missing values
print(df.isnull().sum())

In [ ]:
# Quick check for numeric values
print(df.describe())

There are no missing values so we can proceed with the next step of looking at and chaing the data types of features if needed

In [ ]:
print(df.dtypes)

In [ ]:
# Looking at the distribution of the target variable

plt.hist(df["G3"], bins=20, edgecolor="k")
plt.xlabel("Final Math Grade (G3)")
plt.ylabel("Number of Students")
plt.title("Distribution of Final Math Grades")
plt.show()

In [ ]:
df.head()

In [ ]:
# I am explicitly defining columns and its types for my convenience

target = "G3"


binary_cols = [
    "schoolsup",
    "famsup",
    "paid",
    "activities",
    "nursery",
    "higher",
    "internet",
    "romantic",
]

ordinal_cols = [
    "age",
    "Medu",
    "Fedu",
    "traveltime",
    "studytime",
    "failures",
    "famrel",
    "freetime",
    "goout",
    "Dalc",
    "Walc",
    "health",
    "absences",
    "G1",
    "G2",
]

nominal_cols = [
    "school",
    "sex",
    "address",
    "famsize",
    "Pstatus",
    "Mjob",
    "Fjob",
    "reason",
    "guardian",
]

# explicitly defining the dataset and the predictor variable
predictors = binary_cols + ordinal_cols + nominal_cols
data = df[predictors + [target]].copy()

In [ ]:
# mapping binaries to yes/no
yesno_map = {"yes": 1, "no": 0, "Yes": 1, "No": 0}
for col in binary_cols:
    data[col] = data[col].map(yesno_map).astype("int8")

In [ ]:
# ordinals + binaries + target -- to be used in correlation
num = data[ordinal_cols + binary_cols + [target]]

In [ ]:
# Pearson correlations
corr_p = num.corr(method="pearson")

# Sort by absolute correlation with G3
cw = corr_p["G3"].drop("G3")
order = cw.abs().sort_values(ascending=False).index
corr_with_G3 = pd.DataFrame({"corr": cw.loc[order], "abs_corr": cw.abs().loc[order]})
print(corr_with_G3.head(15))

### Pearson correlation

In [ ]:
# creating heatmap of top correlates
topk = order[:10].tolist() + ["G3"]
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.heatmap(
    corr_p.loc[topk, topk],
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    vmin=-1,
    vmax=1,
    square=True,
)
plt.title("Pearson correlation (top features vs G3)")
plt.tight_layout()
plt.show()

We can see very strong correlations for G1 and G2 with G3, which makes sense since prior grades would dominate and be a good predictor of the final grade. Students who have more past failures also have a tendency to perform worse (-0.36). Interestingly, mother's education does have a small positive effect (0.22). Lifestyle factors such as going out with friends, being in a relationship, or having longer travel times show only weak negative links, while father’s education and alcohol consumption appear to have little effect.

### Building Random Forest Model

I am going to perform a Random Forest regressor for this dataset since it would be able to model nonlinear relationships and interactions. It also conventiently handles small datasets without requiring heavy hyperparameter tuning. 

However, before running the Random Forest regressor, I am going to intentionally remove G1 and G2 from the main model to remove the 'prior math grade predicts final math grade' dynamics and instead focus on the students' habits and behaviours. 

In [ ]:
# removing G1 and G2
predictors = [col for col in predictors if col not in ["G1", "G2"]]

# one-hot encoding nominal categories
X = pd.get_dummies(data[predictors], columns=nominal_cols, drop_first=True)
y = data[target].astype(float)

print("Shape after encoding:", X.shape)
print("Shape of target variable", y.shape)

In [ ]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [ ]:
# Fitting RF model
rf = RandomForestRegressor(
    n_estimators=600,  # a bit higher for stability
    max_depth=None,  # letting trees grow; small dataset
    min_samples_leaf=2,  # mild regularization
    random_state=42,
    n_jobs=-1,
)
rf.fit(X_train, y_train)

# I used Chatgpt to help me figure the hyperparater tuning

### Doing Permutation importance to find top important features

In [ ]:
# Feature importance values
fi = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

# Show top 15
plt.figure(figsize=(8, 6))
sns.barplot(x=fi.head(15).values, y=fi.head(15).index, palette="viridis")
plt.xlabel("Random Forest Feature Importance")
plt.ylabel("")
plt.title("Top 15 Features Predicting Final Grade (G3)")
plt.tight_layout()
plt.show()

# Also print top 15
print(fi.head(15))

Based on the feature importance plot above, we can see that a student's absences and failures play a relatively important role in their final Math grade.


For my analysis, I am going to choose 'absence' and 'studytime' and examine their effects using three interpretability tools. The first, A one-dimensional Partial Dependence Plot (PDP) will help show me the average effect of a feature on the model’s predictions, marginalizing over all the other features. While PDP is good for getting a global, averaged view, the Individual Conditional Expectation (ICE) will plot the prediction for each individual student as one feature is varied (for instance, absences and studytime), keeping all others fixed. It will essentially help me uncover the interaction effects that PDP averages would hide. The last interpretability tool I will use is Accumulated Local Effects (ALE), which shows the local effect of a feature on predictions. However, unlike the PDP, it will not extrapolate into regions where there is little or missing data, which is helpful in skewed or correlated datasets.

### PDP, ICE, ALE plots

This is taken fro mthe code template provided but adapted (with the help of Chatgpt 5 to fit my use case)

#### PDP for 'absences'

In [ ]:
# pick the absences column
feature_name = "absences"  # "absences"
feature_index = X.columns.get_loc(feature_name)

# build a sensible grid (use the actual observed unique values, sorted)
# if there are many unique counts, you can thin it with slicing
vals = np.sort(X[feature_name].unique())
feature_values = vals  # or vals[::2] to thin

# Initialize array to store average predictions
average_predictions = np.zeros_like(feature_values, dtype=float)

# Duplicate the dataset to modify feature values
X_modified = X.copy()

# Loop over feature values
for i, value in enumerate(feature_values):
    # Set the chosen feature to the current value for all instances
    X_modified.iloc[:, feature_index] = value

    # Predict using the modified dataset
    preds = rf.predict(X_modified)

    # Calculate average prediction for the current feature value
    average_predictions[i] = preds.mean()

# Plot the partial dependence for the chosen feature (absences)
plt.figure(figsize=(7, 4))
plt.plot(feature_values, average_predictions, linewidth=2)
plt.xlabel("Absences (count)")
plt.ylabel("Average predicted final grade (G3)")
plt.title("Partial Dependence — Absences")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

#### ICE for 'absence'

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
PartialDependenceDisplay.from_estimator(
    rf, X, ["absences"], kind="both", subsample=80, grid_resolution=50, n_jobs=-1, ax=ax
)
plt.title("ICE + PDP — Absences")
plt.tight_layout()
plt.show()

#### ALE for 'absence'

In [ ]:
ax = ale_plot(
    rf,
    X_train,
    "absences",
    bins=10,
    monte_carlo=True,
    monte_carlo_rep=30,
    monte_carlo_ratio=0.5,
)

fig = ax.get_figure()  # get parent figure
fig.set_size_inches(10, 15)  # resize figure
plt.tight_layout()
plt.show()

The curve from the PDP is non-linear. It rises quickly from 0 to around 2–3 absences, then steadily drops and flattens out after about 20 absences. The small rise at very low absences could just be because there aren’t many students with zero absences. After that, the expected pattern shows up — more absences = lower grades. The model predicts higher grades for students with just a few absences, but as absences increase (beyond 3 to 5), predicted grades drop. After around 20 absences, the curve levels off and stays pretty flat at just over 10.5.

This makes sense. Students with low absences generally perform better. As absences go up, performance tends to drop. And once attendance is poor enough, extra absences don’t change much.

The ICE plot shows the same trend (orange line), but we can also see that students differ a lot. Some students’ grades fall sharply with just a few absences. Others stay flat, suggesting absences don’t affect them as much.

So while the PDP gives the average decline, the ICE plot shows that absences matter much more for some students than others.

The ALE plot is also consistent. Students with very few absences tend to do slightly above average, but grades start dropping once absences pass about 5 to 7 days. After 20, the negative effect levels off, so more absences don’t make a big additional difference.

#### PDP for 'studytime'

In [ ]:
# pick the absences column
feature_name = "studytime"  # "absences"
feature_index = X.columns.get_loc(feature_name)

# build a sensible grid (use the actual observed unique values, sorted)
# if there are many unique counts, you can thin it with slicing
vals = np.sort(X[feature_name].unique())
feature_values = vals  # or vals[::2] to thin

# Initialize array to store average predictions
average_predictions = np.zeros_like(feature_values, dtype=float)

# Duplicate the dataset to modify feature values
X_modified = X.copy()

# Loop over feature values
for i, value in enumerate(feature_values):
    # Set the chosen feature to the current value for all instances
    X_modified.iloc[:, feature_index] = value

    # Predict using the modified dataset
    preds = rf.predict(X_modified)

    # Calculate average prediction for the current feature value
    average_predictions[i] = preds.mean()

# Plot the partial dependence for the chosen feature (absences)
plt.figure(figsize=(7, 4))
plt.plot(feature_values, average_predictions, linewidth=2)
plt.xlabel("Study Time")
plt.ylabel("Average predicted final grade (G3)")
plt.title("Partial Dependence — Study Time")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

#### ICE for 'studytime'

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
PartialDependenceDisplay.from_estimator(
    rf,
    X,
    ["studytime"],
    kind="both",
    subsample=80,
    grid_resolution=50,
    n_jobs=-1,
    ax=ax,
)
plt.title("ICE + PDP — Studytime")
plt.tight_layout()
plt.show()

#### ALE for 'studytime'

In [ ]:
# 1D ALE: Absences
ax = ale_plot(
    rf,
    X_train,
    "studytime",
    bins=10,
    monte_carlo=True,
    monte_carlo_rep=30,
    monte_carlo_ratio=0.5,
)

fig = ax.get_figure()  # get parent figure
fig.set_size_inches(10, 15)  # resize figure
plt.tight_layout()
plt.show()

From the PDP plot, I can see that average predicted grades stay flat when studytime is around 1–2 hours, but then jump up sharply once studytime goes beyond 2. The more studytime, the higher the predicted grade. This makes sense — more time studying usually means better understanding of the material.

The ICE plot confirms the same overall pattern (orange line), but also shows how much students differ. Some students get a big boost in predicted grades as they study more, while others barely improve. So the effect of studytime isn’t the same for everyone.

I can see a similar trend from the ALE plot but with a smoother curve. Instead of a sudden jump at 2, it shows a more gradual rise as studytime increases. This happens because ALE corrects for correlations (students who study more may also differ on other things)- thus avoiding the the sharp jump that is visible in the PDP and giving it a cleaner view of the effect.

Overall, we can evidently see that more studytime is linked to better performance. PDP shows the sharp average jump, ICE shows that not all students benefit equally, and ALE shows the gradual upward slope once we account for correlations.

So far, the PDP, ICE, and ALE plots for absences and studytime on their own give us a good idea of how each feature affects predicted grades individually. But in reality, these factors don’t act in isolation — how much time a student studies might matter differently depending on how often they miss class. To capture this kind of interaction, we need to look at both features together. Thus, in the setion below, I also create the 2D ALE plot to understand the dynamics of both my chosen features.

### 2D ALE plot for absence and studytime

In [ ]:
ax = ale_plot(rf, X_train, ["absences", "studytime"], bins=10, monte_carlo=True)
fig = ax.get_figure()
fig.set_size_inches(8, 6)
plt.tight_layout()
plt.show()

From the 2D ALE plot, I see that students with high absences and low studytime do the worst (blue area). On the flip side, students with higher studytime and only moderate absences do better (red area). Either extreme is bad — very high absences or very low studytime both push grades down.

The 1D ALE plots already showed this: more absences hurt, more studytime helps. The 2D version just makes clear how the two interact.

One thing that stands out is the very top-left corner (low absences + very high studytime). It shows a neutral or even slightly negative effect. That’s probably because there aren’t many students in this group, so the estimates get noisy. It’s similar to what we saw in the PDP for absences, where the tiny bump at zero absences was likely due to very few students falling into that category.

Overall, the plots clearly show that, both absences and studytime, play an important role in predicting grades. Low absences and higher studytime are linked to better performance, while skipping too many classes or barely studying drags grades down. The PDPs gave the big picture averages, the ICE plots showed how students differ, and the ALE plots helped clean up the effects and reveal the interactions. Simply put, showing up and putting in the study hours really does matter.